# Face recognition model

## Import required libraries


In [ ]:
import json
import zipfile
import os
import numpy as np
import statistics
import cv2
import torch

import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential, save_model, load_model

import keras 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.optimizers import RMSprop
from keras import backend as K
from keras.preprocessing import image

import matplotlib.pyplot as plt

## Download Kaggle datasets

In [ ]:
!mkdir .kaggle

token = {"username":"arbenamusa","key":"2e2a2c05c2bbf4867a1d729b6cb9a1e8"}
with open('/content/.kaggle/kaggle.json','w') as file:
    json.dump(token,file)

!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}

Mund të kërkojmë dhe listojmë datasete specifike të cilat ekzistojnë në Kaggle.


> ```
!kaggle datasets list -s face
```

> -s është shkurtes për search.

In [ ]:
!kaggle datasets download -d dansbecker/5-celebrity-faces-dataset -p/content/ziped_dataset

In [ ]:
dataset = "/content/ziped_dataset/5-celebrity-faces-dataset.zip"

with zipfile.ZipFile(dataset, 'r') as z:
    z.extractall('/content/datasets/5-celebrity-faces-dataset') 

## Build the model

In [ ]:
train_data_dir = '/content/datasets/5-celebrity-faces-dataset/data/train'
validation_data_dir = '/content/datasets/5-celebrity-faces-dataset/data/val'
test_data_dir = '/content/datasets/5-celebrity-faces-dataset/val'

In [ ]:
img_width, img_height = 224, 224

train_samples = 93
validation_samples = 25

epochs = 20
batch_size = 16
numclasses = 5
labels = ['ben_afflek',  'elton_john',  'jerry_seinfeld',  'madonna',  'mindy_kaling']

### Image preprocessing 

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1. / 255,
                                   rotation_range = 10,
                                   zoom_range = 0.1,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   vertical_flip = False,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (img_width, img_height),
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical')
validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        target_size = (img_width, img_height),
                                                        batch_size = batch_size,
                                                        class_mode = 'categorical')

In [ ]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [ ]:
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.
    return img

### Model

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape = input_shape)
flatten_layer = keras.layers.Flatten(input_shape = base_model.output_shape[1:])
relu_dense_layer = Dense(4096, activation = 'relu')
dropout_layer = Dropout(0.5)
relu_dense_layer2 = Dense(4096, activation = 'relu')
softmax_dense_layer = Dense(numclasses, activation = 'softmax')

In [ ]:
top_model = Sequential()
top_model.add(flatten_layer)
top_model.add(relu_dense_layer)
top_model.add(dropout_layer)
top_model.add(relu_dense_layer2)
top_model.add(dropout_layer)
top_model.add(softmax_dense_layer)

model = Model(inputs = base_model.input, outputs = top_model(base_model.output))

In [ ]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(learning_rate = 1e-5, decay = 1e-7),
              metrics = ['accuracy'])

### Train

In [ ]:
modelHistory = model.fit(train_generator,
                         steps_per_epoch = train_samples // batch_size,
                         epochs = epochs,
                         validation_data = validation_generator,
                         validation_steps = validation_samples // batch_size)


Mund të ruajmë peshat e modelit

```
model.save_weights('model_weights.h5')
```

ose tërë modelin.



```
model.save('model.h5')
```





#### Model training history

In [ ]:
validation_loss = modelHistory.history['val_loss']
training_loss = modelHistory.history['loss']

epoch_count = range(1, len(validation_loss) + 1)
fig=plt.figure(figsize = (12, 4))

fig.add_subplot(121)
plt.plot(epoch_count, validation_loss, 'm-')
plt.plot(epoch_count, training_loss, 'b-')
plt.legend(['Humbjet e validimit', 'Humbjet e trajnimit'])
plt.xlabel('Epoka')
plt.ylabel('Humbja')

val_acc = modelHistory.history['val_accuracy']
training_acc = modelHistory.history['accuracy']

epoch_count = range(1, len(val_acc) + 1)

fig.add_subplot(122)
plt.plot(epoch_count, val_acc, 'm-')
plt.plot(epoch_count, training_acc, 'b-')
plt.legend(['Saktësia e validimit', 'Saktësia e trajnimit'])
plt.xlabel('Epoka')
plt.ylabel('Saktësia')

plt.show()

print("Numri i epokave {}.".format(epochs))
print("Mesatarja e saktësisë nëpër epoka {:.2f}%".format(statistics.mean(val_acc)*100))
print("Minimumi i saktësië nëpër epoka {:.2f}%".format(min(val_acc)*100))
print("Maksimumi i saktësië nëpër epoka {:.2f}%".format(max(val_acc)*100))

### Model Evaluation

In [ ]:
!mkdir /content/tests
!mkdir /content/tests/ben_afflek
!mkdir /content/tests/elton_john
!mkdir /content/tests/jerry_seinfeld
!mkdir /content/tests/madonna
!mkdir /content/tests/mindy_kaling

In [ ]:
def show_predicted_image(imag, path):
    image.save_img("/content/tests/" + path, imag[0])

    model_prediction = model(imag)
    model_result = np.squeeze(model_prediction)
    max_result = np.argmax(model_result)

    img = cv2.imread("/content/tests/" + path, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.axis('off')
    plt.title('{} : {:.2f}% '.format(labels[max_result], model_result[max_result]*100))
    plt.imshow(img)
    plt.show()

In [ ]:
img_path = os.path.join(test_data_dir, 'madonna/httpassetsrollingstonecomassetsarticlemadonnadavidbowiechangedthecourseofmylifeforeversmallsquarexmadonnabowiejpg.jpg')
input_image = preprocess_image(img_path4)

show_predicted_image(input_image, "madonna/madonna.png")

# Fast Gradient Sign Method

In [ ]:
def FGSM(clean_image, epsilon, label, targeted):

    model_prediction = model(clean_image)
    loss_object = tf.keras.losses.CategoricalCrossentropy()

    target_label = tf.one_hot(label, model_prediction.shape[-1])
    target_label = tf.reshape(target_label, (1, model_prediction.shape[-1]))

    tensor_image = tf.convert_to_tensor(clean_image)

    with tf.GradientTape() as tape:
        tape.watch(tensor_image)
        prediction = model(tensor_image)
        loss = loss_object(target_label, prediction)

    gradient = tape.gradient(loss, tensor_image)
    perturbation = tf.sign(gradient)
    
    if(targeted):
      adversarial_image = clean_image - epsilon*perturbation
    else:
      adversarial_image = clean_image + epsilon*perturbation
    
    adversarial_image = tf.clip_by_value(adversarial_image, -1, 1)

    return adversarial_image

In [ ]:
epsilon = 0.04
adversarial_image_targeted = FGSM(input_image, epsilon, 0, True)
adversarial_image_untargeted = FGSM(input_image, epsilon, 3, False)


